In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/Reviews.csv')

In [4]:
#lower, strip, remove punctuation, word tokenize
import re
lst_rv = []
for rv in data['Text']:
    lst_rv.append(re.sub('[^a-z ]', '', rv.lower().strip()).split())

In [5]:
#remove stop-words
from nltk.corpus import stopwords

lst_stop_words = stopwords.words('English')
for i in range(len(lst_stop_words)):
    lst_stop_words[i] = re.sub('[^a-z ]', '', lst_stop_words[i]).lower()

for rv_index in range(len(lst_rv)):
    new_rv = []
    for word in lst_rv[rv_index]:
        if word not in lst_stop_words:
            new_rv.append(word)
    lst_rv[rv_index] = new_rv

: 

: 

In [ ]:
#lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

lemmatizer = WordNetLemmatizer()

for rv_index in range(len(lst_rv)):
    lst_rv[rv_index] = [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in nltk.pos_tag(lst_rv[rv_index])]

In [ ]:
#find phrases(min_count, threshold = hyperparameter)
import gensim

bigram = gensim.models.Phrases(lst_rv, min_count = 5, threshold = 100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

for rv_index in range(len(lst_rv)):
    lst_rv[rv_index] =  bigram_mod[lst_rv[rv_index]]

In [ ]:
#doc2bow
from gensim import corpora

id2word = corpora.Dictionary(lst_rv)

corpus = [id2word.doc2bow(rv) for rv in lst_rv]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 3),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 2)],
 [(13, 2),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 2),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1)],
 [(35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 2),
  (68, 2),
  (69, 1),
  (70, 1),
  (71, 1)],
 [(4, 1),
  (7, 1),
  (10, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1)],
 [(71, 1),
  (86, 

In [ ]:
df = pd.DataFrame(data={'Corpus': corpus, 'Score': data['Score']})
df

,Corpus,Score
0,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...",5
1,"[(13, 2), (20, 1), (21, 1), (22, 1), (23, 1), ...",1
2,"[(35, 1), (36, 1), (37, 1), (38, 1), (39, 1), ...",4
3,"[(4, 1), (7, 1), (10, 1), (72, 1), (73, 1), (7...",2
4,"[(71, 1), (86, 1), (87, 1), (88, 1), (89, 2), ...",5
...,...,...
568449,"[(4, 1), (7, 2), (89, 1), (106, 1), (130, 1), ...",5
568450,"[(1, 1), (2, 1), (31, 1), (76, 2), (146, 1), (...",2
568451,"[(3, 3), (6, 3), (9, 1), (16, 1), (31, 1), (68...",5
568452,"[(3, 1), (7, 1), (68, 2), (102, 1), (130, 1), ...",5


In [ ]:
df.to_csv('clean_data.csv')

In [ ]:
clean_data = pd.read_csv('clean_data.csv')

In [ ]:
clean_data.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
corpus = []
for rv in clean_data['Corpus']:
    rv_corpus = []
    o_index = 0
    while o_index < rv.rindex('('):
        o_index = rv.index('(', o_index + 1)
        e_index = rv.index(')', o_index)
        sub_text = rv[o_index+1:e_index].split(', ')
        rv_corpus.append(tuple([int(sub_text[0]), int(sub_text[1])]))
    corpus.append(rv_corpus)

In [21]:
corpus[105132]

[(2, 1),
 (4, 1),
 (6, 8),
 (10, 1),
 (12, 2),
 (20, 1),
 (38, 1),
 (77, 2),
 (97, 1),
 (130, 2),
 (143, 2),
 (144, 3),
 (153, 1),
 (159, 1),
 (189, 1),
 (190, 1),
 (210, 1),
 (211, 1),
 (239, 1),
 (241, 1),
 (252, 1),
 (279, 1),
 (302, 1),
 (312, 1),
 (422, 1),
 (457, 1),
 (466, 1),
 (491, 1),
 (532, 1),
 (571, 1),
 (625, 1),
 (673, 1),
 (682, 1),
 (796, 1),
 (815, 2),
 (895, 1),
 (1016, 1),
 (1046, 1),
 (1169, 1),
 (1177, 1),
 (1180, 1),
 (1199, 1),
 (1203, 1),
 (1220, 2),
 (1268, 1),
 (1338, 1),
 (1431, 2),
 (1450, 1),
 (1619, 1),
 (1776, 1),
 (2299, 1),
 (2426, 1),
 (2678, 1),
 (2861, 1),
 (4100, 1),
 (4321, 1),
 (4726, 1),
 (5561, 1),
 (5746, 2),
 (7359, 1),
 (7678, 1),
 (10002, 1),
 (10891, 1),
 (11000, 1),
 (14659, 1),
 (15946, 1),
 (21369, 1),
 (26955, 1),
 (66216, 1),
 (100001, 1),
 (100002, 1)]

In [19]:
for rv in corpus:
    for word in rv:
        if word[0] > 100000:
            print(corpus.index(rv))
            break 

105132
105133
105138
105146
105149
105152
105154
105157
105165
105168
105171
105175
105176
105182
105190
105191
105192
105196
105197
105198
105201
105202
105203
105205
105207
105208
105209
105210
105215
105216
105217
105219
105222
105226
105229
105230
105233
105234
105238
105243
105249
105260
105261
105262
105266
105268
105271
105277
105455
105458
105461
105462
105466
105467
105468
105472
105519
105520
105523
105532
105533
105544
105546
105548
105554
105557
105559
105567
105570
105571
105588
105593
105594
105608
105609
105611
105612
105613
105615
105616
105618
105620
105621
105624
105625
105627
105633
105639
105641
105644
105645
105648
105650
105651
105653
105654
105658
105660
105661
105666
105668
105672
105675
105676
105681
105683
105690
105692
105695
105697
105698
105699
105700
105702
105704
105705
105708
105710
105713
105714
105717
105718
105719
105721
105722
105723
105724
105728
105733
105734
105737
105742
105745
105748
105751
105752
105755
105758
105760
105767
105773
105774
105781

KeyboardInterrupt: 